In [1]:
import gurobipy as gp
import pandas as pd
from gurobipy import GRB


In [59]:
production   = ['Baltimore','Cleveland','Little Rock','Birmingham','Charleston']
distribution = ['Columbia','Indianapolis','Lexington','Nashville','Richmond','St_Louis']

In [60]:
transp_cost = pd.read_csv('cost.csv')

In [61]:
transp_cost 

,production,distribution,cost
0,Baltimore,Columbia,4.50
1,Baltimore,Indianapolis,5.09
2,Baltimore,Lexington,4.33
3,Baltimore,Nashville,5.96
4,Baltimore,Richmond,1.96
5,Baltimore,St_Louis,7.30
6,Cleveland,Columbia,2.43
7,Cleveland,Indianapolis,2.37
8,Cleveland,Lexington,2.54
9,Cleveland,Nashville,4.13


In [62]:
transp_cost = transp_cost.pivot(index='production',columns='distribution',values='cost')

In [63]:
max_prod = pd.Series([180,200,140,80,180], index = production, name="max_production")
n_demand = pd.Series([89,95,121,101,116,181], index = distribution, name="demand")


In [64]:
frac = 0.75

In [65]:
m = gp.Model('widgets')
x = {}

for p in production:
    for d in distribution:
        x[p,d] = m.addVar(name = p+"_to_"+d)

m.update()

In [66]:
m = gp.Model('widgets')


x = m.addVars(production,distribution, name='prod_ship')

m.update()

In [21]:
x

{('Baltimore', 'Columbia'): <gurobi.Var prod_ship[Baltimore,Columbia]>,
 ('Baltimore', 'Indianapolis'): <gurobi.Var prod_ship[Baltimore,Indianapolis]>,
 ('Baltimore', 'Lexington'): <gurobi.Var prod_ship[Baltimore,Lexington]>,
 ('Baltimore', 'Nashville'): <gurobi.Var prod_ship[Baltimore,Nashville]>,
 ('Baltimore', 'Richmond'): <gurobi.Var prod_ship[Baltimore,Richmond]>,
 ('Baltimore', 'St.Louis'): <gurobi.Var prod_ship[Baltimore,St.Louis]>,
 ('Cleveland', 'Columbia'): <gurobi.Var prod_ship[Cleveland,Columbia]>,
 ('Cleveland', 'Indianapolis'): <gurobi.Var prod_ship[Cleveland,Indianapolis]>,
 ('Cleveland', 'Lexington'): <gurobi.Var prod_ship[Cleveland,Lexington]>,
 ('Cleveland', 'Nashville'): <gurobi.Var prod_ship[Cleveland,Nashville]>,
 ('Cleveland', 'Richmond'): <gurobi.Var prod_ship[Cleveland,Richmond]>,
 ('Cleveland', 'St.Louis'): <gurobi.Var prod_ship[Cleveland,St.Louis]>,
 ('Little Rock', 'Columbia'): <gurobi.Var prod_ship[Little Rock,Columbia]>,
 ('Little Rock',
  'Indianapolis'): 

In [67]:
meet_demand = m.addConstrs((gp.quicksum(x[p,d] for p in production) <= n_demand[d] for d in distribution), name='meet_demand')

m.update()
meet_demand

{'Columbia': <gurobi.Constr meet_demand[Columbia]>,
 'Indianapolis': <gurobi.Constr meet_demand[Indianapolis]>,
 'Lexington': <gurobi.Constr meet_demand[Lexington]>,
 'Nashville': <gurobi.Constr meet_demand[Nashville]>,
 'Richmond': <gurobi.Constr meet_demand[Richmond]>,
 'St_Louis': <gurobi.Constr meet_demand[St_Louis]>}

In [68]:
can_produce  = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) <= max_prod[p] for p in production), name='can_produce')
must_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) >= frac*max_prod[p] for p in production), name='must_produce')
m.update()

In [40]:
transp_cost['Columbia'].loc['Baltimore']

4.5

In [45]:
production

['Baltimore', 'Cleveland', 'Little Rock', 'Birmingham', 'Charleston']

In [53]:
distribution[5]

'St.Louis'

In [57]:
transp_cost['St. Louis']

production
Baltimore      7.30
Birmingham     4.01
Charleston     4.60
Cleveland      4.88
Little Rock    2.92
Name: St. Louis, dtype: float64

In [51]:
transp_cost

distribution,Columbia,Indianapolis,Lexington,Nashville,Richmond,St. Louis
production,,,,,,
Baltimore,4.50,5.09,4.33,5.96,1.96,7.30
Birmingham,3.33,4.33,3.38,1.53,5.95,4.01
Charleston,3.02,2.61,1.61,4.44,2.36,4.60
Cleveland,2.43,2.37,2.54,4.13,3.20,4.88
Little Rock,6.42,4.83,3.39,4.40,7.44,2.92


In [69]:
for p in production:
    for d in distribution:
        print(p,d)
        transp_cost[d].loc[p]

Baltimore Columbia
Baltimore Indianapolis
Baltimore Lexington
Baltimore Nashville
Baltimore Richmond
Baltimore St_Louis
Cleveland Columbia
Cleveland Indianapolis
Cleveland Lexington
Cleveland Nashville
Cleveland Richmond
Cleveland St_Louis
Little Rock Columbia
Little Rock Indianapolis
Little Rock Lexington
Little Rock Nashville
Little Rock Richmond
Little Rock St_Louis
Birmingham Columbia
Birmingham Indianapolis
Birmingham Lexington
Birmingham Nashville
Birmingham Richmond
Birmingham St_Louis
Charleston Columbia
Charleston Indianapolis
Charleston Lexington
Charleston Nashville
Charleston Richmond
Charleston St_Louis


In [70]:
m.setObjective(gp.quicksum(transp_cost[d].loc[p]*x[p,d] for p in production for d in distribution), GRB.MINIMIZE)

In [71]:
m.write('widget_shipment.lp')

In [72]:
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 16 rows, 30 columns and 90 nonzeros
Model fingerprint: 0xf8f89186
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 2e+02]
Presolve removed 5 rows and 0 columns
Presolve time: 0.02s
Presolved: 11 rows, 35 columns, 65 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   7.312500e+01   0.000000e+00      0s
      10    1.3022500e+03   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.04 seconds (0.00 work units)
Optimal objective  1.302250000e+03


In [77]:
x_values = pd.Series(m.getAttr('X',x), name = "shipment", index = transp_cost.index)
sol = pd.concat([transp_cost, x_values], axis=1)

In [79]:
all_vars = {v.varName: v.x for v in m.getVars()}

In [80]:
all_vars

{'prod_ship[Baltimore,Columbia]': 19.0,
 'prod_ship[Baltimore,Indianapolis]': 0.0,
 'prod_ship[Baltimore,Lexington]': 0.0,
 'prod_ship[Baltimore,Nashville]': 0.0,
 'prod_ship[Baltimore,Richmond]': 116.0,
 'prod_ship[Baltimore,St_Louis]': 0.0,
 'prod_ship[Cleveland,Columbia]': 69.0,
 'prod_ship[Cleveland,Indianapolis]': 81.0,
 'prod_ship[Cleveland,Lexington]': 0.0,
 'prod_ship[Cleveland,Nashville]': 0.0,
 'prod_ship[Cleveland,Richmond]': 0.0,
 'prod_ship[Cleveland,St_Louis]': 0.0,
 'prod_ship[Little Rock,Columbia]': 0.0,
 'prod_ship[Little Rock,Indianapolis]': 0.0,
 'prod_ship[Little Rock,Lexington]': 0.0,
 'prod_ship[Little Rock,Nashville]': 0.0,
 'prod_ship[Little Rock,Richmond]': 0.0,
 'prod_ship[Little Rock,St_Louis]': 105.0,
 'prod_ship[Birmingham,Columbia]': 0.0,
 'prod_ship[Birmingham,Indianapolis]': 0.0,
 'prod_ship[Birmingham,Lexington]': 0.0,
 'prod_ship[Birmingham,Nashville]': 60.0,
 'prod_ship[Birmingham,Richmond]': 0.0,
 'prod_ship[Birmingham,St_Louis]': 0.0,
 'prod_ship[Ch